### 문맥을 예측해서 다음 단어 예측해보기

In [19]:
text = """경마장에 있는 말이 뛰고 있다\n
그의 말이 법이다\n
가는 말이 고와야 오는 말이 곱다\n"""

In [20]:
from keras_preprocessing.text import Tokenizer
t = Tokenizer()
t.fit_on_texts([text])
encoded = t.texts_to_sequences([text])[0]

In [21]:
vocab_size = len(t.word_index) + 1 

# 케라스 토크나이저의 정수 인코딩은 인덱스가 1부터 시작하지만
# 케라스 원-핫 인코팅에서 배열의 인덱스가 0 부터 시작하기 떄문에
# 배열의 크기를 실제 단어 집합의 크기보다 +1 로 생성해야 하므로 미리 +1 서언
print("단어 집합의 크기 : %d" % vocab_size)

단어 집합의 크기 : 12


In [22]:
print(t.word_index)

{'말이': 1, '경마장에': 2, '있는': 3, '뛰고': 4, '있다': 5, '그의': 6, '법이다': 7, '가는': 8, '고와야': 9, '오는': 10, '곱다': 11}


In [25]:
sequences = list()
for line in text.split('\n') : # \n 을 기준으로 문장 토큰화
    encoded = t.texts_to_sequences([line])[0]
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)
        
print("훈련데이터개수 : %d" % len(sequences))

훈련데이터개수 : 11


In [26]:
print(sequences)

[[2, 3], [2, 3, 1], [2, 3, 1, 4], [2, 3, 1, 4, 5], [6, 1], [6, 1, 7], [8, 1], [8, 1, 9], [8, 1, 9, 10], [8, 1, 9, 10, 1], [8, 1, 9, 10, 1, 11]]


In [27]:
print(max(len(i) for i in sequences))

6


In [28]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
sequences = pad_sequences(sequences, maxlen=6, padding = "pre")

In [29]:
print(sequences)

[[ 0  0  0  0  2  3]
 [ 0  0  0  2  3  1]
 [ 0  0  2  3  1  4]
 [ 0  2  3  1  4  5]
 [ 0  0  0  0  6  1]
 [ 0  0  0  6  1  7]
 [ 0  0  0  0  8  1]
 [ 0  0  0  8  1  9]
 [ 0  0  8  1  9 10]
 [ 0  8  1  9 10  1]
 [ 8  1  9 10  1 11]]


In [30]:
import numpy as np
sequences = np.array(sequences)
x = sequences[:,:-1]
y = sequences[:,-1]

# 리스트의 마지막 열을 제외하고 저장한 것은 x
# 리스트의 마지막 열만 저장한 것은 y

In [31]:
from tensorflow.keras.utils import to_categorical
y = to_categorical(y, num_classes = vocab_size)

In [32]:
print(y)

[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


In [34]:
from tensorflow.keras.layers import Embedding, Dense , SimpleRNN
from tensorflow.keras.models import Sequential

model = Sequential()
# Y를 제거하였으므로 이제는 X 길이는 5
model.add(Embedding(vocab_size, 10, input_length=5))
model.add(SimpleRNN(32))
model.add(Dense(vocab_size, activation="softmax"))
model.compile(loss = "categorical_crossentropy", optimizer ="adam",
             metrics = ["accuracy"])
model.fit(x,y,epochs=200, verbose =2)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Epoch 1/200
11/11 - 0s - loss: 2.4797 - acc: 0.1818
Epoch 2/200
11/11 - 0s - loss: 2.4674 - acc: 0.2727
Epoch 3/200
11/11 - 0s - loss: 2.4552 - acc: 0.3636
Epoch 4/200
11/11 - 0s - loss: 2.4429 - acc: 0.3636
Epoch 5/200
11/11 - 0s - loss: 2.4304 - acc: 0.4545
Epoch 6/200
11/11 - 0s - loss: 2.4177 - acc: 0.4545
Epoch 7/200
11/11 - 0s - loss: 2.4046 - acc: 0.4545
Epoch 8/200
11/11 - 0s - loss: 2.3911 - acc: 0.4545
Epoch 9/200
11/11 - 0s - loss: 2.3771 - acc: 0.4545
Epoch 10/200
11/11 - 0s - loss: 2.3625 - acc: 0.4545
Epoch 11/200
11/11 - 0s - loss: 2.3473 - acc: 0.4545
Epoch 12/200
11/11 - 0s - loss: 2.3314 - acc: 0.4545
Epoch 13/200
11/11 - 0s - loss: 2.3148 - acc: 0.3636
Epoch 14/200
11/11 - 0s - loss: 2.2973 - acc: 0.3636
Epoch 15/200
11/11 - 0s - loss:

Epoch 142/200
11/11 - 0s - loss: 0.2530 - acc: 1.0000
Epoch 143/200
11/11 - 0s - loss: 0.2476 - acc: 1.0000
Epoch 144/200
11/11 - 0s - loss: 0.2422 - acc: 1.0000
Epoch 145/200
11/11 - 0s - loss: 0.2371 - acc: 1.0000
Epoch 146/200
11/11 - 0s - loss: 0.2320 - acc: 1.0000
Epoch 147/200
11/11 - 0s - loss: 0.2271 - acc: 1.0000
Epoch 148/200
11/11 - 0s - loss: 0.2223 - acc: 1.0000
Epoch 149/200
11/11 - 0s - loss: 0.2176 - acc: 1.0000
Epoch 150/200
11/11 - 0s - loss: 0.2130 - acc: 1.0000
Epoch 151/200
11/11 - 0s - loss: 0.2086 - acc: 1.0000
Epoch 152/200
11/11 - 0s - loss: 0.2043 - acc: 1.0000
Epoch 153/200
11/11 - 0s - loss: 0.2001 - acc: 1.0000
Epoch 154/200
11/11 - 0s - loss: 0.1960 - acc: 1.0000
Epoch 155/200
11/11 - 0s - loss: 0.1920 - acc: 1.0000
Epoch 156/200
11/11 - 0s - loss: 0.1881 - acc: 1.0000
Epoch 157/200
11/11 - 0s - loss: 0.1843 - acc: 1.0000
Epoch 158/200
11/11 - 0s - loss: 0.1806 - acc: 1.0000
Epoch 159/200
11/11 - 0s - loss: 0.1771 - acc: 1.0000
Epoch 160/200
11/11 - 0s - l

In [37]:
def sentence_generation(model, t, current_word, n): # 모델 토크나이저, 현재 단어, 반복획수
    init_word = current_word  # 처음 들어온 단어도 마지막에 같이 출력하기 위해 저장
    sentence = ''
    for _ in range(n):
        encoded = t.texts_to_sequences([current_word])[0]
        encoded = pad_sequences([encoded], maxlen=5, padding="pre")
        result = model.predict_classes(encoded, verbose=0)
        
        for word, index in t.word_index.items():
            if index == result: # 만약 예측한 단어와 인덱스가 동일한 단어가 있다면
                break
        current_word = current_word + ' ' + word 
        sentence = sentence + ' ' + word # 예측단어를 문장에 저장
        
    sentence = init_word + sentence
    return sentence
                
    

In [38]:
print(sentence_generation(model,t,"경마장에",4))

경마장에 있는 말이 뛰고 있다


In [39]:
print(sentence_generation(model, t, "그의", 2))

그의 말이 법이다
